# **Project UAS Kecerdasan Buatan**

## **Library**

In [1]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from xgboost import XGBClassifier
import optuna
from imblearn.over_sampling import SMOTE

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## **Data**

In [2]:
data = pd.read_csv('adult.csv')
data.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [3]:
X = data.drop('income', axis=1)
y = data['income']

## **EDA**

In [4]:
data[data == '?'] = np.nan
data.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,NaN,77053,HS-grad,9,Widowed,NaN,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,NaN,186061,Some-college,10,Widowed,NaN,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [5]:
data.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education.num      int64
marital.status    object
occupation        object
relationship      object
race              object
sex               object
capital.gain       int64
capital.loss       int64
hours.per.week     int64
native.country    object
income            object
dtype: object

## **Preprocessing**

### **Train Test Split**

In [6]:
X_train, X_val, y_train, y_val = train_test_split(data.drop('income', axis=1), data['income'], test_size=0.2, random_state=42)

### **Encoding**

In [7]:
# Encode categorical features in X_train and X_val
categorical_cols = X_train.select_dtypes(include=['object']).columns

le_dict = {}
for col in categorical_cols:
    le = LabelEncoder()
    # Fit on all data (train + val) to handle unseen labels
    all_values = pd.concat([X_train[col], X_val[col]], axis=0).astype(str)
    le.fit(all_values)
    X_train[col] = le.transform(X_train[col].astype(str))
    X_val[col] = le.transform(X_val[col].astype(str))
    le_dict[col] = le

# Encode target variable if needed
target_le = LabelEncoder()
y_train = target_le.fit_transform(y_train)
y_val = target_le.transform(y_val)

### **Handling Missing Values**

In [8]:
data.isnull().sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     583
income               0
dtype: int64

In [9]:
print(data["occupation"].value_counts())

occupation
Prof-specialty       4140
Craft-repair         4099
Exec-managerial      4066
Adm-clerical         3770
Sales                3650
Other-service        3295
Machine-op-inspct    2002
Transport-moving     1597
Handlers-cleaners    1370
Farming-fishing       994
Tech-support          928
Protective-serv       649
Priv-house-serv       149
Armed-Forces            9
Name: count, dtype: int64


In [10]:
data["workclass"].replace(np.nan, "Private", inplace=True)
data["occupation"].replace(np.nan, "Sales", inplace=True)
data["native.country"].replace(np.nan, "United-States", inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_4796\2765059703.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["workclass"].replace(np.nan, "Private", inplace=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_4796\2765059703.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

In [11]:
train_data = X_train.copy()
train_data['income'] = y_train

majority_class = train_data[train_data['income'] == 0]
minority_class = train_data[train_data['income'] == 1]

minority_upsampled = resample(minority_class, replace=True, n_samples=len(majority_class), random_state=42)

balanced_train_data = pd.concat([majority_class, minority_upsampled])

X_train_balanced = balanced_train_data.drop('income', axis=1)
y_train_balanced = balanced_train_data['income']

In [15]:
balanced_train_data['income'].value_counts()

income
0    19744
1    19744
Name: count, dtype: int64

### **Scaling**

In [24]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_balanced)
X_val_scaled = scaler.transform(X_val)

## **Modelling**

In [ ]:
model_dt = DecisionTreeClassifier(random_state=42)
model_dt.fit(X_train_scaled, y_train_balanced)
y_pred_dt = model_dt.predict(X_val_scaled)

print("Decision Tree Accuracy:", accuracy_score(y_val, y_pred_dt.round()))

Decision Tree Accuracy: 0.8185168125287886


In [22]:
model_knn = KNeighborsClassifier()
model_knn.fit(X_train_scaled, y_train_balanced)
y_pred_knn = model_knn.predict(X_val_scaled)

print("KNN Accuracy:", accuracy_score(y_val, y_pred_knn.round()))
print()

KNN Accuracy: 0.7724550898203593



## **Optimization**

### **Genetic Algorithm**

### **Particle Swarm Optimization**

##  **Evaluation**